In [1]:
import csv
import numpy as np
import torch


f =  open("C:/Users/OWO/Documents/AA_CODE/@Projects/Projects/ai_0004_lc_Crop/Crop_recommendation.csv")
read = csv.reader(f)

#data to list
data = []
for row in read:
  data.append(row)

#changing str to float
x_train_data = [[float(data[row+1][col]) for col in range(len(data[0])-1)] for row in range(len(data[:])-1)]#-1 in col is for the last crop data +-1 in row is for the first row legend
#print(x_train_data)

#getting y train data
y_train_data = [[data[row+1][len(data[0])-1] for col in range(1)] for row in range(len(data[:])-1)]
#print(y_train_data)

#counting kinds
def counting_kinds(list):
  list.sort()
  temp = list[0][0]
  cnt = 0
  kindlist = [[cnt,temp]]
  for i in range(len(list)):
    if list[i][0] != temp:
      temp = list[i][0]
      cnt = cnt + 1
      kindlist.append([cnt,temp])
  return kindlist
def kinds_to_num(ylist,kindlist):
  list_train = [[0 for col in range(22)] for row in range(len(ylist))]
  for i in range(len(ylist)):
    for j in range(len(kindlist)):
      if ylist[i][0] == kindlist[j][1]:
        list_train[i][j] = 1
  return list_train



print(counting_kinds(y_train_data))#its 22 kinds of crops
kindlist = counting_kinds(y_train_data)
y_train_data = [[data[row+1][len(data[0])-1] for col in range(1)] for row in range(len(data[:])-1)]
#print(y_train_data)
y_ones_train= kinds_to_num(y_train_data,kindlist)##this is the one with 22 col and 2200 row 
#print(len(y_ones_train))
f.close()





##DATA

x_train_data = torch.Tensor(x_train_data)
y_ones_train = torch.Tensor(y_ones_train)

print(x_train_data.size())
print(y_ones_train.size())

[[0, 'apple'], [1, 'banana'], [2, 'blackgram'], [3, 'chickpea'], [4, 'coconut'], [5, 'coffee'], [6, 'cotton'], [7, 'grapes'], [8, 'jute'], [9, 'kidneybeans'], [10, 'lentil'], [11, 'maize'], [12, 'mango'], [13, 'mothbeans'], [14, 'mungbean'], [15, 'muskmelon'], [16, 'orange'], [17, 'papaya'], [18, 'pigeonpeas'], [19, 'pomegranate'], [20, 'rice'], [21, 'watermelon']]
torch.Size([2200, 7])
torch.Size([2200, 22])


# Hareware

In [2]:
##Hardwere
import torch
if torch.cuda.is_available() == True:
  device = 'cuda'
  templist = [1,2,3]
  templist = torch.FloatTensor(templist).to(device)
  print("Cude torch working : ",end="")
  print(templist.is_cuda)
  print("current device no. : ",end="")
  print(torch.cuda.current_device())
  print("GPU device count : ",end="")
  print(torch.cuda.device_count())
  print("GPU name : ",end="")
  print(torch.cuda.get_device_name(0))
  print("device : ",device)
else:
  print("cant use gpu , activating cpu")
  device = 'cpu'

Cude torch working : True
current device no. : 0
GPU device count : 1
GPU name : NVIDIA GeForce GTX 1080
device :  cuda


# ML

In [3]:
import torch

#Hardware
print("GPU Status ",end="")
print("Working Well") if torch.cuda.is_available() else print("Not good, Activating CPU calculation")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

x_train_data.to(device=device)
y_ones_train.to(device=device)

feature = 7
nb_class = 22


#lr_list = [1,1e-1,1e-2,1e-3,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9]
lr_list = [1e-2]
for LR in lr_list:

  W = torch.zeros((feature,nb_class),requires_grad=True)
  b = torch.zeros(22,requires_grad=True)

  #set optimizer
  optimizer = torch.optim.SGD([W,b],lr=LR)
  nb_epochs = 10000
  
  #learning
  for epoch in range(nb_epochs +1 ):
    #hypothesis = torch.nn.functional.softmax(torch.matmul(x_train_data,W) + b) #turns out sigmoid is better
    hypothesis = torch.sigmoid(torch.matmul(x_train_data,W) + b)

    #cost = (y_ones_train * -torch.log(torch.nn.functional.softmax(hypothesis,dim=1))).sum(dim=1).mean()
    loss = torch.nn.BCELoss()
    cost = loss(hypothesis,y_ones_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
  hypothesis_tolist = hypothesis.tolist()
  #for i in range(2200):
    #print("in LR :",LR,"data :",i,"is",kindlist[hypothesis_tolist[i].index(max(hypothesis_tolist[i]))][1],"with %.2f"%max(hypothesis_tolist[i])*100,"%"+" COST : %.3f"%cost.item())
    #print("In LR : %.0E data : %d is %s with %.2f "%" COST : %.3f"%(LR,i,kindlist[hypothesis_tolist[i].index(max(hypothesis_tolist[i]))][1],max(hypothesis_tolist[i])*100,cost.item()))
  print("In LR : %.0E the cost is : %f"%(LR,cost.item()))
  

GPU Status Working Well
In LR : 1E-02 the cost is : 0.058110


# LR
In LR : 1E+00 the cost is : 4.545455

In LR : 1E-01 the cost is : 1.848365

In LR : 1E-02 the cost is : 0.074471

In LR : 1E-03 the cost is : 0.106367

In LR : 1E-04 the cost is : 0.161250

In LR : 1E-05 the cost is : 0.231938

In LR : 1E-06 the cost is : 0.520766

In LR : 1E-07 the cost is : 0.670983

In LR : 1E-08 the cost is : 0.690871

In LR : 1E-09 the cost is : 0.692919

# Data evaluation

In [4]:
prediction = torch.argmax(hypothesis,dim=1)#argmax returns the INDEX #dim = 0 means max in column // dim = 1 means max in row

evaluation = torch.argmax(y_ones_train,dim=1) == prediction
print("Accuracy : ",end="")
print(evaluation.sum().item()/len(evaluation)*100,"%")

Accuracy : 92.36363636363636 %


# Result Program

In [6]:
print("Wellcome to match crop\n you will input 7 features of your environment and we will match your best crop")

dialog = ["input the ratio of Nitrogen content in soil","input the ratio of Phosphorous content in soil","input the ratio of Potassium content in soil","input the temperature in degree Celsius","input the relative humidity in %","input the ph value of the soil","input rainfall in mm"]
new_train_data = []

for i in range(7):
  print(dialog[i])
  new_train_data.append(float(input()))
#print(new_train_data)


new_train_data = torch.Tensor(new_train_data)

hypothesis = torch.sigmoid(torch.matmul(new_train_data,W) + b)
print(" We recommend you : ",end="")
print(kindlist[torch.argmax(hypothesis).item()][1])


Wellcome to match crop
 you will input 7 features of your environment and we will match your best crop
input the ratio of Nitrogen content in soil
input the ratio of Phosphorous content in soil
input the ratio of Potassium content in soil
input the temperature in degree Celsius
input the relative humidity in %
input the ph value of the soil
input rainfall in mm
 We recommend you : rice


# End

now... its 2023/01/19 0033i


It took a long time because I was stuck on a lot of things, like the CSV file. 

It was hard to make a proper list, and it was a bit unorganized because I was trying to figure out how to convert a string to a number list.

The new thing I realized I was missing was that I had applied the softmax function to my hypothesis twice, which resulted in only 60% accuracy. 

Nonetheless, I fixed it.

The most difficult part was definitely the CSV. But through it all, I learned a lot